--------------------------------------------------------------------

Fully-Connected Neural Network

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from keras.optimizers import Adam
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy

import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.model_selection import train_test_split
def trainModel(model, epochs, optimizer, X_train, y_train):
    batch_size = 128
    model.compile(optimizer=optimizer, loss=SparseCategoricalCrossentropy(), metrics=['accuracy'])
    return model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)

def plotTrainingHistory(history):
    print("Training accuracy:", max(history.history['accuracy']))
    pd.DataFrame(history.history).plot(figsize=(12, 6))
    plt.show()


30sec Dataset FNN 

In [ ]:
# train_data = pd.read_csv('/kaggle/input/3trainsec/features_3_sec_train.csv')
# train_data = train_data.iloc[:, 2:]  # Exclude the first column if it's just an index
# y_train = train_data['label']  # genre variable
# X_train = train_data.loc[:, train_data.columns != 'label']  # Select all columns except the labels

# # Normalize X_train
# cols = X_train.columns
# min_max_scaler = preprocessing.MinMaxScaler()
# np_scaled = min_max_scaler.fit_transform(X_train)
# X_train = pd.DataFrame(np_scaled, columns=cols)

# # Load the test data from "features_3_sec_test.csv"
# test_data = pd.read_csv('/kaggle/input/3testsec/features_3_sec_test.csv')
# test_data = test_data.iloc[:, 2:]  # Exclude the first column if it's just an index
# y_test = test_data['label']  # genre variable
# X_test = test_data.loc[:, test_data.columns != 'label']  # Select all columns except the labels

# # Normalize X_test
# np_scaled = min_max_scaler.transform(X_test)
# X_test = pd.DataFrame(np_scaled, columns=cols)


# le = LabelEncoder()
# y_train = le.fit_transform(y_train)
# y_test = le.transform(y_test)


data = pd.read_csv('/kaggle/input/30secdata/features_30_sec (1).csv')
data = data.iloc[:, 2:]
y = data['label']  
X = data.loc[:, data.columns != 'label']

# Normalize X
cols = X.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
X = pd.DataFrame(np_scaled, columns=cols)

# Encode labels using LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),

    Dense(256, activation='relu'),
    Dropout(0.2),

    Dense(64, activation='relu'),
    Dropout(0.2),

    Dense(10, activation='softmax')
])
print(model.summary())
adam_optimizer = Adam()

# Compile the model
model.compile(optimizer=adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model on the training set
model_history = trainModel(model=model, epochs=300, optimizer=adam_optimizer, X_train=X_train, y_train=y_train)

# Evaluating the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Predictions on the test set
y_pred = model.predict(X_test)

# Convert one-hot encoded labels back to categorical labels
y_test_labels = np.argmax(y_test, axis=1)
y_pred_labels = np.argmax(y_pred, axis=1)

# Compute precision, recall, and F1 score
precision = precision_score(y_test_labels, y_pred_labels, average='weighted')
recall = recall_score(y_test_labels, y_pred_labels, average='weighted')
f1 = f1_score(y_test_labels, y_pred_labels, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

In [ ]:
test_embeddings = model.predict(X_test)

# Calculate cosine similarity
cosine_sim = cosine_similarity(test_embeddings)

# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cosine_sim, annot=True, cmap='viridis', fmt=".2f", xticklabels=False, yticklabels=False)
plt.title('Cosine Similarity Heatmap')
plt.show()

In [ ]:
from keras.layers import Dense, Dropout
from keras.models import Sequential, Model
from sklearn.metrics.pairwise import cosine_similarity
layer_index = -1

# ... (your existing model definition)

# Create an intermediate model for getting layer outputs
intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[layer_index].output)

# Get embeddings for training and test data
X_train_embeddings = intermediate_layer_model.predict(X_train)
X_test_embeddings = intermediate_layer_model.predict(X_test)

# Compute cosine similarity matrix
similarity_matrix = cosine_similarity(X_test_embeddings, X_train_embeddings)

# Get the indices of most similar samples and their corresponding similarity scores
most_similar_indices = similarity_matrix.argmax(axis=1)
similarity_scores = similarity_matrix.max(axis=1)

# Display the results
for i, (test_index, similarity_score) in enumerate(zip(range(len(X_test)), similarity_scores)):
    train_index = most_similar_indices[i]
    print(f"Test sample {test_index} is most similar to train sample {train_index} with a similarity score of {similarity_score}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# ... (your existing code to compute the cosine similarity matrix)

# Plot the cosine similarity matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(similarity_matrix, cmap='viridis', annot=True, fmt=".2f", xticklabels=False, yticklabels=False)
plt.title('Cosine Similarity Matrix')
plt.xlabel('Training Set Samples')
plt.ylabel('Test Set Samples')
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Assuming X_test and y_test are your test data and labels

# Predict probabilities for each class
y_pred_probabilities = model.predict(X_test)

# Convert probabilities to class predictions
y_pred_classes = np.argmax(y_pred_probabilities, axis=1)

# If your labels are one-hot encoded, convert them to classes
if len(y_test.shape) > 1:
    y_true_classes = np.argmax(y_test, axis=1)
else:
    y_true_classes = y_test

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

# Calculate precision, recall, and F1 score for each class
precision = np.diag(conf_matrix) / np.sum(conf_matrix, axis=0)
recall = np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print results for each class
for i in range(len(precision)):
    print(f"Class {i}:")
    print(f"  Precision: {precision[i]}")
    print(f"  Recall: {recall[i]}")
    print(f"  F1 Score: {f1_score[i]}")

# You can also use sklearn's classification report for a summary
classification_report_str = classification_report(y_true_classes, y_pred_classes)
print("Classification Report:\n", classification_report_str)


In [ ]:
# Define and compile the model
input_shape = X_train.shape[1:]  # Adjust according to your input data shape
num_classes = len(np.unique(y))  # Assuming y contains class labels
combined_model = build_combined_model(input_shape)
combined_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
batch_size = 32
epochs = 10
combined_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

# Evaluate the model on the test set
eval_metrics = combined_model.evaluate(X_test, y_test)
print("Test Loss:", eval_metrics[0])
print("Test Accuracy:", eval_metrics[1])